In [ ]:
from fbprophet import Prophet
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer  # make_scorer importieren
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd

# Lade die Daten
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Bereite die Trainings- und Testdaten wie zuvor vor (80% Training, 20% Test)
train_size = int(len(data) * 0.8)
train = data.iloc[:train_size]
test = data.iloc[train_size:]

# Prophet benötigt zwei Hauptspalten: ds (Datum) und y (Zielvariable)
train_prophet = pd.DataFrame({
    'ds': pd.to_datetime(train[['year', 'month', 'day']]),  # Kombiniere Jahr, Monat, Tag zu einem Datum
    'y': train['Sales']  # Zielvariable
})

# Zusätzliche Regressoren, die nicht automatisch in Prophet integriert sind
# (bereits encodierte Features wie 'Open', 'Promo', 'promo2', lag features, etc.)
additional_regressors = ['Open', 'Promo', 'promo2', 'lag_1', 'lag_7']

# Prophet Modell mit zusätzlichem exogenen Regressor (z.B. für 'Promo')
model = Prophet()

# Hinzufügen der zusätzlichen Regressoren
for reg in additional_regressors:
    model.add_regressor(reg)

# Trainiere das Modell auf den Trainingsdaten
model.fit(train_prophet[['ds', 'y'] + additional_regressors])

# Erstelle den Test-Datensatz im passenden Format für Prophet
test_prophet = pd.DataFrame({
    'ds': pd.to_datetime(test[['year', 'month', 'day']]),
    'y': test['Sales']  # Dies ist nur für die Evaluierung
})

# Füge die Regressoren auch dem Testdatensatz hinzu
for reg in additional_regressors:
    test_prophet[reg] = test[reg]

# Vorhersagen machen
forecast = model.predict(test_prophet[['ds'] + additional_regressors])

# Prophet gibt viele zusätzliche Spalten zurück; wir interessieren uns für 'yhat' (die Vorhersagen)
y_pred_prophet = forecast['yhat']

# Berechne den RMSPE für die Testdaten
test_rmspe_prophet = rmspe(test['Sales'], y_pred_prophet)
print("Test RMSPE (Prophet):", test_rmspe_prophet)


In [ ]:
from prophet import Prophet
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer  # make_scorer importieren
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np

# Lade die Daten
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Bereite die Trainings- und Testdaten wie zuvor vor (80% Training, 20% Test)
train_size = int(len(data) * 0.8)
train = data.iloc[:train_size]
test = data.iloc[train_size:]

# 1. Bereite die Trainingsdaten für Prophet vor: Kombiniere year, month und day zu einem Datum
train_prophet = pd.DataFrame({
    'ds': pd.to_datetime(train[['year', 'month', 'day']]),  # Erstelle das Datum
    'y': train['Sales']  # Zielvariable
})

# 2. Zusätzliche Regressoren hinzufügen: Lag-Features, Open, Promo, Promo2
additional_regressors = ['Open', 'Promo', 'promo2', 'lag_1', 'lag_7']

# 3. Prophet-Modell erstellen
model = Prophet()

# 4. Zusätzliche Regressoren zu Prophet hinzufügen
for reg in additional_regressors:
    model.add_regressor(reg)

# 5. Modell auf den Trainingsdaten anpassen
model.fit(train_prophet[['ds', 'y'] + additional_regressors])

# 6. Bereite die Testdaten im selben Format für Prophet vor
test_prophet = pd.DataFrame({
    'ds': pd.to_datetime(test[['year', 'month', 'day']]),  # Datum im gleichen Format wie im Training
    'y': test['Sales']  # Zielvariable (für spätere Bewertung)
})

# 7. Zusätzliche Regressoren in die Testdaten einfügen
for reg in additional_regressors:
    test_prophet[reg] = test[reg]

# 8. Vorhersagen auf den Testdaten machen
forecast = model.predict(test_prophet[['ds'] + additional_regressors])

# 9. Prophet gibt viele zusätzliche Spalten zurück; wir interessieren uns nur für 'yhat' (die Vorhersagen)
y_pred_prophet = forecast['yhat']

# 10. Definiere die RMSPE-Funktion (wie im ursprünglichen Code)
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# 11. RMSPE für das Testset berechnen
test_rmspe_prophet = rmspe(test['Sales'], y_pred_prophet)
print("Test RMSPE (Prophet):", test_rmspe_prophet)


In [8]:
from prophet import Prophet
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer  # make_scorer importieren
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np

# Erstelle Lag-Variablen
data['lag_1'] = data['Sales'].shift(1)
data['lag_7'] = data['Sales'].shift(7)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Bereite die Trainings- und Testdaten wie zuvor vor (80% Training, 20% Test)
train_size = int(len(data) * 0.8)
train = data.iloc[:train_size]
test = data.iloc[train_size:]

# Bereite die Trainingsdaten für Prophet vor
train_prophet = pd.DataFrame({
    'ds': pd.to_datetime(train[['year', 'month', 'day']]),  # Erstelle das Datum
    'y': train['Sales']  # Zielvariable
})

# Zusätzliche Regressoren hinzufügen
additional_regressors = ['Open', 'Promo', 'promo2', 'lag_1', 'lag_7']

# Sicherstellen, dass die Regressoren in den Trainingsdaten vorhanden sind
for reg in additional_regressors:
    train_prophet[reg] = train[reg]

# Prophet-Modell erstellen
model = Prophet()

# Zusätzliche Regressoren zu Prophet hinzufügen
for reg in additional_regressors:
    model.add_regressor(reg)

# Nullwerte in den Lag-Variablen entfernen
train_prophet = train_prophet.dropna(subset=additional_regressors)

# Modell auf den Trainingsdaten anpassen
model.fit(train_prophet[['ds', 'y'] + additional_regressors])

# Bereite die Testdaten für Prophet vor
test_prophet = pd.DataFrame({
    'ds': pd.to_datetime(test[['year', 'month', 'day']]),  # Datum im gleichen Format wie im Training
    'y': test['Sales']  # Zielvariable (für spätere Bewertung)
})

# Zusätzliche Regressoren in die Testdaten einfügen
for reg in additional_regressors:
    test_prophet[reg] = test[reg]

# Nullwerte in den Testdaten behandeln (falls notwendig)
test_prophet = test_prophet.dropna(subset=additional_regressors)

# Vorhersagen auf den Testdaten machen
forecast = model.predict(test_prophet[['ds'] + additional_regressors])

# Prophet gibt viele zusätzliche Spalten zurück; wir interessieren uns nur für 'yhat'
y_pred_prophet = forecast['yhat']
# 10. Definiere die RMSPE-Funktion (wie im ursprünglichen Code)
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# 11. RMSPE für das Testset berechnen
test_rmspe_prophet = rmspe(test['Sales'], y_pred_prophet)
print("Test RMSPE (Prophet):", test_rmspe_prophet)

11:28:39 - cmdstanpy - INFO - Chain [1] start processing
11:35:42 - cmdstanpy - INFO - Chain [1] done processing


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).